Using my models! I'm going to provide two options for trying out each model. If you have GPU access, you can use the first option. If you don't, you can use the second option. The first option will be faster, but the second option will still work. I'll provide instructions for both.

Git Diff Cleaner: First I clean a diff using this code block (put your diff into the mydiff variable and keep it under 2000 characters)

In [ ]:
def extract_added_lines(diff, max_length=2000):
    added_lines = []
    for line in diff.split('\n'):
        if line.startswith('+') and not line.startswith('+++'):
            added_line = line[1:]  # Remove the '+' sign
            if sum(len(l) + 1 for l in added_lines) + len(added_line) <= max_length:
                added_lines.append(added_line)
            else:
                return None
                break
    return '\n'.join(added_lines)

In [ ]:
mydiff = """
<your diff here>
"""

In [ ]:
clean_diff = extract_added_lines(mydiff)

Now this is the classifier model! Run the first block if you are on colab or have a GPU. Run the second block if you are on your own computer.

In [ ]:
#If you have GPU access, this cell is for you! (Maybe you can try it on Google Colab)
!pip install transformers
from transformers import pipeline

pipe = pipeline("text-classification", model="davidgaofc/TechDebtClassifier")

result = pipe(clean_diff)

result

In [ ]:
#If you are trying to inference on your own computer, this cell is for you! You will need an api key from huggingface with read access to the model.
import requests
API_KEY = "put your huggingface api_key here"
API_URL = "https://api-inference.huggingface.co/models/davidgaofc/TechDebtClassifier"
headers = {"Authorization": f"Bearer {API_KEY}"}
data = {"inputs": clean_diff}

response = requests.post(API_URL, headers=headers, json=data)
result = response.json()
result

Now you have classified the code! LABEL_1 means that the code is technical debt, and LABEL_0 means that the code is not technical debt.

If it has a LABEL_1 score higher than 0.5, maybe you want to generate text that describes it using the second model.

In [ ]:
#If you have GPU access, this cell is for you! (Maybe you can try it on Google Colab)
from transformers import pipeline

pipe = pipeline("text2text-generation", model="davidgaofc/TechDebtLabeler")

result = pipe(clean_diff)

result

In [ ]:
#If you are trying to inference on your own computer, this cell is for you!
API_URL = "https://api-inference.huggingface.co/models/davidgaofc/TechDebtLabeler"
headers = {"Authorization": f"Bearer {API_KEY}"}
data = {"inputs": clean_diff}

response = requests.post(API_URL, headers=headers, json=data)
result = response.json()

result

Great! Take these results with a grain of salt, of course. The models are still a work in progress. But I hope you find them cool!